Jensen Shannon Divergence + AgglomerativeClustering

In [ ]:
def JensenShannon_similarity(df,time_list,st_list,cal_col):
      '''
      [함수 사용법]
        X축에 들어갈 변수를 input_list 인덱싱 0번 자리(맨앞)에 둘 것
        그 뒤로 기준을 잡고 싶은 변수들을 차례로 넣어 주면 됩니다.
        from scipy.spatial import distance 꼭 할것!
      '''


      make_amt= df.groupby(time_list+st_list)[cal_col].sum().reset_index()

      '''pivot 만들기'''
      amt_sum = make_amt.groupby(st_list)[cal_col].sum().reset_index(name='cal_col_sum')
      amt_sum = amt_sum[amt_sum['cal_col_sum'] != 0] #JSD를 구할때 0이 있으면 문제가 생김
      make_amt = make_amt.merge(amt_sum,on=st_list,how='left')
      make_amt['cal_col_prob'] = make_amt[cal_col]/make_amt['cal_col_sum']

      # pivot_table을 활용하여 long to wide 작업 실시
      amt_pivot= pd.pivot_table(make_amt,
                                index=st_list,
                                columns=time_list,
                                values='cal_col_prob').reset_index()
      amt_pivot.fillna(0,inplace=True)

      #피벗된 연도별 amt를 사용해 고객간의 유사도를 구해보자.
      amt_array=np.array(amt_pivot.drop(st_list,axis=1))

      js_similarity =[]
      for row_r in range(amt_array.shape[0]):       
        dt_list=[]
        for row_c in range(amt_array.shape[0]):
            #dt= js_divergence(amt_pivot.iloc[row_r,3:].values,amt_pivot.iloc[row_c,3:].values)
            '''
            js_divergence(만든 함수)로 하면 0값을 계산을 못함. 
            sqrt(js_divergence)인 distance.jensenshannon 함수 사용
            '''
            dt = distance.jensenshannon(amt_array[row_r,:],amt_array[row_c,:],2.0)
            dt_list.append(dt)
        js_similarity.append(dt_list)

      js_frame =pd.DataFrame(np.array(js_similarity))
      js_similarity = pd.concat([amt_pivot,js_frame],axis=1)

      return amt_sum[st_list],js_similarity

In [ ]:
def make_clustering(gp_df,time_list,st_list,cal_col,cluster_name,split_num=2):
    st_series,df = JensenShannon_similarity(gp_df,time_list,st_list,cal_col)
    df_s = df.iloc[:,-df.shape[0]:] # 클러스터링용 dataframe 생성
    df_s.fillna(0,inplace=True)

    #클러스터링
    clustering = AgglomerativeClustering(n_clusters=int(df_s.shape[0]/split_num),
                                      affinity='precomputed',
                                      linkage='average')
    clustering.fit(df_s)
    
    return_df = pd.DataFrame(st_series)         #pd.DataFrame(DataFrame) 오류X

    print(return_df.shape,len(clustering.labels_))

    return_df[cluster_name] = clustering.labels_
    return return_df